# Hydro-Solar Energy Data Exploration

In [1]:
# 2018.03  J. Kaelin 

In [2]:
# import libraries
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
from decimal import *  #for formatting decimals

In [3]:
# set up plotly in 'offline' mode                            #ToDo JK: this should be cleaned up and documented
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
#print (__version__) # requires version >= 1.9.0

# collect all plotly setups here
import plotly.offline as plotly  #testing
from plotly.graph_objs import *  #testing

In [4]:
import scipy
from scipy import signal
#import cufflinks as cf
from scipy.stats import variation 

In [5]:
# set wd for this procedure and project 
import sys, os
#os.chdir("/home/kaelin_joseph/TunnelGIS")

## Demand Curve for Electricity in Laos

In [6]:
# electricity demand curve for Laos (as example)
# Power System Development Plan for Lao PDR, 2004, Manusell
# Dec value (last value in array) corrected to agree with figure text and with Thailand data
demand=array([125, 135, 140, 138, 135, 125, 125, 120, 125, 130, 125, 125])  #MW
#print('Mean demand power: '+str(demand.mean())+' MW')

# reference targeted demand for a hydro-solar installation
demand_target = 100  #MW

# adjust demand curve to targeted demand
# return adjusted monthly energy demand as 'demand_monthly_energy'
def adjust_demand_monthly_energy(demand, demand_target):
    demand_adj = demand * demand_target / demand.mean()  #MW
    print('Reference mean demand: '+str(round(mean(demand_adj),1))+' MW')
    demand_monthly_energy = demand_adj * 8760.0 /12 /1000  #GWh
    return demand_monthly_energy

demand_monthly_energy = adjust_demand_monthly_energy(demand, demand_target)  #GWh/month
print('Reference mean monthly demand: '+str(round(mean(demand_monthly_energy.mean()),1))+' GWh/month')

Reference mean demand: 100.0 MW
Reference mean monthly demand: 73.0 GWh/month


## Solar Irradiation Data

In [7]:
# daily solar irradiation 

# import dataFrame with calculated results from Tunnel GIS
solar_data_df = pd.read_csv('SolarIrradiation.NamAng.2004.nasa.csv',index_col=0)

# solar irradiation data for NT1 site in Laos
# Daily Average Insolation Incident On A Horizontal Surface (kWh/m^2/day
# https://eosweb.larc.nasa.gov/cgi-bin/sse/daily.cgi?email=skip@larc.nasa.gov
# NT1  18.3 N 104.1 E
# Nam Ang  15.101806 N 107.125897 E
# 2004
# import download in openoffice spreadshhet and save as csv

# comparisons
#  http://pvwatts.nrel.gov/pvwatts.php  -> Bangkok, Thailand -> 4.91 kWh/m2/day horizontal, no loss
#  https://solargis.com/maps-and-gis-data/download/laos -> Global Horizontal Irradiation -> -> 4.7 kWh/m2/day

In [8]:
print(len(solar_data_df.index))
solar_data_df.head()

366


,MO,DY,swv_dwn
YEAR,,,
2004,1,1,5.49
2004,1,2,4.87
2004,1,3,4.99
2004,1,4,5.39
2004,1,5,5.43


In [9]:
# monthly solar irradiation 

# horizontal solar irradiation 22 year mean from nasa data (kWh/m2/day)
# Southern Laos  18N, 104E  #1983-2005  #annual mean = 4.64 kWh/m2/day
# https://asdc-arcgis.larc.nasa.gov/sse/ -> Data Download -> horizontal surface -> ascii
irr_daily_mean_mo = array([4.62, 4.94, 5.40, 5.57, 4.87, 3.98, 3.86, 3.97, 4.56, 4.70, 4.68, 4.50])

In [10]:
# print solar irradiation data statistics and plot data

irr = solar_data_df['swv_dwn'].tolist()

trace1 = go.Scatter(
        y=signal.savgol_filter(irr, 53, 3),  #https://plot.ly/python/smoothing/
        x=arange(1, 365, 1),
        name= 'solar 2004',
        )

trace2 = go.Scatter(
        y = irr_daily_mean_mo,
        x=arange(15, 380, 30),
        name= 'solar mean',
        )

layout=go.Layout(title="Horizontal Solar Irradiation - Southern Laos", 
                 xaxis={'title':'day'},
                 yaxis={'title':'kWh/m2/day'},
                 width=700,
                 height=400,
                )

figure=go.Figure(data=[trace1, trace2],layout=layout)

print(' ')
print('Annual mean 2004 of daily solar irradiation on horizontal surface: ' 
       +str(round(solar_data_df['swv_dwn'].mean(),1))+' kWh/m2/day')
print('Standard deviation 2004 of daily solar irradiation on horizontal surface: ' 
       +str(round(solar_data_df['swv_dwn'].std(),1))+' kWh/m2/day')

plotly.iplot(figure)

 
Annual mean 2004 of daily solar irradiation on horizontal surface: 4.6 kWh/m2/day
Standard deviation 2004 of daily solar irradiation on horizontal surface: 1.3 kWh/m2/day


Wet season in 2004 was late compared to average of 1983 to 2005 

## PV Solar Energy

In [11]:
# solar energy

# solar installed capacity
# installed capacity = 1kW/m2 * panel area * eff  #kW
# eff = efficiency is assumed as 0.18 for solar cells and 0.8 for balance of electical system, etc.
# PV energy from titled module assumed comparable to energy from horizontal modules 
solar_installed = 80  #MW

In [12]:
# solar energy generation 

# mean monthly solar energy = mean daily solar irradiation * installed capacity * days/month * eff  #kWh
def solar_mean_monthly(solar_installed):
    mean = irr_daily_mean_mo * solar_installed * 365/12.0 /1000  #GWh
    return mean
print('Installed capacity: '+str(solar_installed)+' MW')
print('Solar monthly mean: '+str(round(solar_mean_monthly(solar_installed).mean(),1))+' GWh/month')
print('Standard deviation of solar monthly mean: '+str(round(solar_mean_monthly(solar_installed).std(),1))+' GWh/month')
print('Coeff of variation of solar monthly mean: '+str(round(variation(solar_mean_monthly(solar_installed)),3)))

# define reference monthly electricity demand curve
demand_target = 15  #MW
demand_monthly_energy = adjust_demand_monthly_energy(demand, demand_target)  #GWh/month
print('Reference mean monthly demand: '+str(round(mean(demand_monthly_energy.mean()),1))+' GWh/month')

#plot results
trace1 = go.Scatter(
        y= solar_mean_monthly(solar_installed),
        # horizontal solar irradiation 22 year mean from nasa data
        # 18N, 104E  #1983-2005  #annual mean = 4.64 kWh/m2/day
        #   for comparison annual mean for directsolar irradtion = 5.97 kWh/m2/day
        # https://asdc-arcgis.larc.nasa.gov/sse/ -> Data Download -> horizontal surface -> ascii
        x=arange(15, 380, 30),
        line=dict(color = 'orange'),
        name= 'solar mean',
        )

trace2 = go.Scatter(
        y= demand_monthly_energy,
        x=arange(15, 380, 30),
        line=dict(color = 'red'),    
        name= 'demand',
        )

layout=go.Layout(title="Solar Generation with Installed Capacity of "+str(solar_installed)+" MW - Southern Laos",
                 xaxis={'title':'day'},
                 yaxis={'title':'GWh/month'},
                 width=700,
                 height=400,
                )

figure=go.Figure(data=[trace1, trace2],layout=layout)
plotly.iplot(figure)

Installed capacity: 80 MW
Solar monthly mean: 11.3 GWh/month
Standard deviation of solar monthly mean: 1.2 GWh/month
Coeff of variation of solar monthly mean: 0.11
Reference mean demand: 15.0 MW
Reference mean monthly demand: 11.0 GWh/month


## Hydropower

In [13]:
# installed hydroelectric plant
project = 'Nam Ang'
rated_hydro_cap = 55  #MW

# monthly discharge data for Nam Ang is from "Hydropower Plants in Southern Lao PDR", August 2017
# comparable discharge data is obtainable using TRMM or MERRA data and catchment areas from GIS,
# a runoff factor of 0.5 is typical for Laos

# discharge_mean=array([3.5, 2.5, 2.0, 2.0, 3.0, 5.05, 13.0, 23.0, 22.5, 15.05, 9.0, 5.05])
# limited to rated discharge = 18.6 m3/s
discharge_mean=array([3.5, 2.5, 2.0, 2.0, 3.0, 5.05, 13.0, 18.6, 18.6, 15.05, 9.0, 5.05])  

# hydro energy
# hydro power = discharge m3/s * 9.805 * net head m * eff  #kW
head_net = 337.7
hydro_mean_power = discharge_mean * 9.805 * head_net * 0.9 /1000  #MW
#print('hydro_mean_power :'+str(hydro_mean_power))

# hydro energy generation mean monthly = mean power * 8760/12 h/mo  #GWh/month
hydro_mean_monthly = hydro_mean_power * 8760/12.0 /1000  #GWh/month

print('Net head: '+str(head_net)+' m')
print('Mean discharge: '+str(round(discharge_mean.mean(),1))+' m3/s')
print('Rated power:')
print('Mean hydro power monthly= '+str(round(hydro_mean_power.mean(),1))+' MW')
print('Mean hydro energy production monthly= '+str(round(hydro_mean_monthly.mean(),1))+' GWh')

Net head: 337.7 m
Mean discharge: 8.1 m3/s
Rated power:
Mean hydro power monthly= 24.2 MW
Mean hydro energy production monthly= 17.6 GWh


In [14]:
# hydroelectric generation

# define reference monthly electricity demand curve
demand_target = 24  #MW
demand_monthly_energy = adjust_demand_monthly_energy(demand, demand_target)  #GWh/month
print('Reference mean monthly demand: '+str(round(mean(demand_monthly_energy.mean()),1))+' GWh/month'); print('')

# required storage to flatten hydro generation over wet and dry seasons
# calculate monthly diff between demand and hdyro generation
hydro_demand_diff = hydro_mean_monthly - demand_monthly_energy
print('Yearly balance of hydro energy minus demand: '+str(round(hydro_demand_diff.sum(),1))+' GWh/year') 

def energy_storage (demand_diff):
    storage = {'fill':[], 'deplete':[]}
    for i in demand_diff:
        if i > 0:
            storage['fill'].append(i)
        else:
            storage['deplete'].append(i)
    #print(storage)
    print('Yearly filling '+str(round(sum(storage['fill'])+1))+' GWh')
    print('Yearly depletion: '+str(round(sum(storage['deplete']),+1))+' GWh') 
    # Volume = Energy * 3600 s/h / (h * 9.805 * eff /10^6)  #Energy in GWh, Volume in m3
    resv_storage = sum(storage['fill']) * 3600 / (head_net * 9.805 * 0.9 /10**6)  #m3
    print('Required reservior storage: '+str(round(resv_storage/10**6,1))+' mil m3'); print('')

energy_storage(hydro_demand_diff)

# plot results 
trace1 = go.Scatter(
        y= hydro_mean_monthly,
        x=arange(15, 380, 30),
        line=dict(color = 'blue'),    
        name= 'hydro mean',
        )

trace2 = go.Scatter(
        y= demand_monthly_energy,
        x=arange(15, 380, 30),
        line=dict(color = 'red'),    
        name= 'demand',
        )

layout=go.Layout(title="Hydro Generation - Monthly Mean - "+str(solar_installed)+" MW - "+project, 
                 xaxis={'title':'day'},
                 yaxis={'title':'GWh/month'},
                 width=700,
                 height=400,
                )

figure=go.Figure(data=[trace1, trace2],layout=layout)

print('Mean monthly discharge = '+str(round(discharge_mean.mean(),1))+' m3/s')
print('Mean hydro energy production monthly= '+str(round(hydro_mean_monthly.mean(),1))+' GWh')
print('Standard deviation of mean energy production monthly = '+str(round(hydro_mean_monthly.std(),1))+' m3/s')
print('Coeff of variation of mean energy production monthly = '+str(round(variation(hydro_mean_monthly),3)))

plotly.iplot(figure)

Reference mean demand: 24.0 MW
Reference mean monthly demand: 17.5 GWh/month

Yearly balance of hydro energy minus demand: 1.5 GWh/year
Yearly filling 78.0 GWh
Yearly depletion: -75.1 GWh
Required reservior storage: 92.6 mil m3

Mean monthly discharge = 8.1 m3/s
Mean hydro energy production monthly= 17.6 GWh
Standard deviation of mean energy production monthly = 13.5 m3/s
Coeff of variation of mean energy production monthly = 0.767


## Hydro-Solar Co-Development

In [15]:
# hdyro-solar energy combined
# Usage:
#   - choose value for 'solar_installed' to roughly balance installed hydro energy (e.g. GWh)
#   - set value for 'demand_target'
#   - adjust value for 'demand_target' 
#     until "Yearly balance of energy generated minus demand" is slightly positive
#   - check required storage

# define reference monthly electricity demand curve
demand_target = 39.5  #MW
demand_monthly_energy = adjust_demand_monthly_energy(demand, demand_target)  #GWh/month
print('Reference mean monthly demand: '+str(round(mean(demand_monthly_energy.mean()),1))+' GWh/month'); print('')

# installed solar chosen to roughly balance day demand with solar and night demand with hydro
solar_installed = 80  #MW
print('Mean solar energy production monthly = '
      +str(round(solar_mean_monthly(solar_installed).mean(),1))+' GWh/month');

# installed hydro capacity
print('Mean hydro energy production monthly= '+str(round(hydro_mean_monthly.mean(),1))+' GWh/month'); print('')

# total power generation (no storage)
total_mean_monthly = solar_mean_monthly(solar_installed) + hydro_mean_monthly

# required storage to flatten hydro generation over wet and dry seasons
# calculate monthly diff between demand and hdyro generation
total_demand_diff = total_mean_monthly - demand_monthly_energy
print('Yearly balance of energy generated minus demand: '+str(round(total_demand_diff.sum(),1))+' GWh/year') 

energy_storage(total_demand_diff)

# plot results

trace1 = go.Scatter(
        y= total_mean_monthly,
        x=arange(15, 380, 30),
        mode='lines',
        #fill = "tozeroy", 
        #line=dict(color = 'blue'),    
        name= 'total',
        )

trace2 = go.Scatter(
        y= hydro_mean_monthly,
        x=arange(15, 380, 30),
        mode='lines',    
        line=dict(
            dash = 'dash',
            #color = 'orange'
            ),        
        name= 'hydro',
        )

trace3 = go.Scatter(
        y= solar_mean_monthly(solar_installed),
        x=arange(15, 380, 30),
        mode='lines',
        line=dict(
            dash = 'dash',
            #color = 'green'
            ),        
        name= 'solar',
        )

trace4 = go.Scatter(
        y= demand_monthly_energy,
        x=arange(15, 380, 30),
        mode='lines',
        fill = "tonexty",
        #fill = "tozeroy""#A020F066" ##,
        ##fillcolor = 'rgba(232, 219, 215, 0.5)', 
        fillcolor = 'rgba(193, 66, 66, 0.1)',
        line=dict(color = 'red',
                  width = 2.5),
        name= 'demand',
        )

layout=go.Layout(title="Total Generation Monthly Mean with "+str(solar_installed)+" MW Installed Solar - "+project, 
                 xaxis={'title':'day'},
                 yaxis={'title':'GWh/month'},
                 width=700,
                 height=400,
                )

figure=go.Figure(data=[trace2, trace3, trace1, trace4],layout=layout)

print('Installed hydro rated capacity = '+ str(rated_hydro_cap)+' MW')
print('Installed solar peak capacity = '+str(solar_installed)+" MW")
print('Monthly energy production = '+str(round(total_mean_monthly.mean(),1))+' GWh')
print('Standard deviation of monthly energy production = '+str(round(total_mean_monthly.std(),1))+' GWh')
print('Coeff of variation of monthly energy production = '+str(round(variation(total_mean_monthly),3)))

plotly.iplot(figure)

# Notes
#   - analysis still based on capped discharge (no cap needed if storage!!)

Reference mean demand: 39.5 MW
Reference mean monthly demand: 28.8 GWh/month

Mean solar energy production monthly = 11.3 GWh/month
Mean hydro energy production monthly= 17.6 GWh/month

Yearly balance of energy generated minus demand: 1.2 GWh/year
Yearly filling 76.0 GWh
Yearly depletion: -73.6 GWh
Required reservior storage: 90.4 mil m3

Installed hydro rated capacity = 55 MW
Installed solar peak capacity = 80 MW
Monthly energy production = 28.9 GWh
Standard deviation of monthly energy production = 12.9 GWh
Coeff of variation of monthly energy production = 0.445


Season variation of combined hydro-solar electrical production decreases already substantially without storage. <br><br>

In [16]:
# hdyro-solar energy combined

# installed solar chosen to roughly balance day demand with solar and night demand with hydro
solar_installed = 300  #MW
print('Mean solar energy production monthly = '
      +str(round(solar_mean_monthly(solar_installed).mean(),1))+' GWh/month');

# installed hydro capacity
print('Mean hydro energy production monthly= '+str(round(hydro_mean_monthly.mean(),1))+' GWh/month'); print('')

# total power generation (no storage)
total_mean_monthly = solar_mean_monthly(solar_installed) + hydro_mean_monthly

# define reference monthly electricity demand curve
demand_target = 85  #MW
demand_monthly_energy = adjust_demand_monthly_energy(demand, demand_target)  #GWh/month
print('Reference mean monthly demand: '+str(round(mean(demand_monthly_energy.mean()),1))+' GWh/month')

# plot results
trace1 = go.Scatter(
        y= total_mean_monthly,
        x=arange(15, 380, 30),
        mode='lines',
        #fill = "tozeroy", 
        #line=dict(color = 'blue'),    
        name= 'total',
        )

trace2 = go.Scatter(
        y= hydro_mean_monthly,
        x=arange(15, 380, 30),
        mode='lines',    
        line=dict(
            dash = 'dash',
            #color = 'orange'
            ),        
        name= 'hydro',
        )

trace3 = go.Scatter(
        y= solar_mean_monthly(solar_installed),
        x=arange(15, 380, 30),
        mode='lines',
        line=dict(
            dash = 'dash',
            #color = 'green'
            ),        
        name= 'solar',
        )

trace4 = go.Scatter(
        y= demand_monthly_energy,
        x=arange(15, 380, 30),
        mode='lines',
        fill = "tonexty",
        #fill = "tozeroy",
        fillcolor ='rgb(234, 211, 204)',
        line=dict(color = 'red'),    
        name= 'demand',
        )

layout=go.Layout(title="Total Generation Monthly Mean with "+str(solar_installed)+" MW Installed Solar - "+project, 
                 xaxis={'title':'day'},
                 yaxis={'title':'GWh/month'},
                 width=700,
                 height=400,
                )

figure=go.Figure(data=[trace2, trace3, trace1, trace4],layout=layout)

print('Installed hydro rated capacity = '+ str(rated_hydro_cap)+' MW')
print('Installed solar peak capacity = '+str(solar_installed)+" MW")
print('Monthly energy production = '+str(round(total_mean_monthly.mean(),1))+' GWh')
print('Standard deviation of monthly energy production = '+str(round(total_mean_monthly.std(),1))+' GWh')
print('Coeff of variation of monthly energy production = '+str(round(variation(total_mean_monthly),3)))

plotly.iplot(figure)

Mean solar energy production monthly = 42.3 GWh/month
Mean hydro energy production monthly= 17.6 GWh/month

Reference mean demand: 85.0 MW
Reference mean monthly demand: 62.1 GWh/month
Installed hydro rated capacity = 55 MW
Installed solar peak capacity = 300 MW
Monthly energy production = 60.0 GWh
Standard deviation of monthly energy production = 11.5 GWh
Coeff of variation of monthly energy production = 0.192


Best seasonal synergy between solar and hydro electrical generation is obtained for large solar installations. <br>
It seems likely that combining a number of solar installations and hydroplants will flatten generation further.

Solar flattens hydro production over the course of a year substantially.<br>
Hydro flattens solar production a smaller amount if installed solar capacity is large compared to hydro.
<br><br>

In [17]:
coeff_var = []
solar_installed_gen = array([0,100,200,300,400,500,600,700,800,900,1000])
for item in solar_installed_gen:
    total_mean_monthly= (solar_mean_monthly(item) + hydro_mean_monthly)
    coeff_var.append(variation(total_mean_monthly))
    
trace1 = go.Scatter(
        y= coeff_var,
        x=solar_installed/rated_hydro_cap,
        name= 'coeff_var',
        )    

layout=go.Layout(title="Coeff of Var of Total Generation Monthly Mean - "+project, 
                 xaxis={'title':'installed solar / installed hydro '},
                 yaxis={'title':'coeff of var'},
                 width=700,
                 height=400,
                )

figure=go.Figure(data=[trace1],layout=layout)
plotly.iplot(figure)